In [1]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS
import inspect
from metafid.ta import RetailInstitutionalPower
from metafid.mfw import DB
from metafid.mfw.deriv import OptionStrategyMFW

In [2]:
ostg_mfw = OptionStrategyMFW(dbname="metafid", user="postgres", pass_=PASS, ua_table="sigma", ostg_table="derivs_optionstrategy", pct_daily_cp=0.3, interval=10)


In [3]:
ostg_mfw.do_job()

Press Ctrl+C to exit


In [2]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)

In [3]:
drop_cols = ["isin","time","open","close","no","volume","low","high","y_final","eps","base_vol","unknown1","unknown2","sector","day_ul","day_ll", "share_no", "mkt_id", "sell_no", "buy_no"]

In [4]:
sigma_df = db.query_all(table="sigma", cols= "(ua,sigma)")

In [5]:
bs = Pricing()
tsetmc = TSETMC(drop_cols=drop_cols)
now = jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
now

'1402-01-17 12:41:46'

In [13]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [6]:
df = tsetmc.option_mv(ua=sigma_df.ua)
def option_type(x):
    return "call" if x.startswith("ض") else "put"
df = df.assign(type=df.option.map(option_type))
df = df.merge(sigma_df, on="ua", how="inner")
df["bs"] = df.apply(lambda x: bs.black_scholes(s_0=x["ua_final"], k=x["strike_price"], t=x["t"],sigma=float(x["sigma"]),type_=x["type"]),axis=1)
df["bs"] = df.bs.fillna(0).apply(lambda x: np.nan if x == np.nan else int(x))
call = df[df.type =="call"]
put = df[df.type =="put"]


/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/option.py:41: RuntimeWarning: invalid value encountered in sqrt
  sigma * np.sqrt(t)
/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/option.py:43: RuntimeWarning: invalid value encountered in sqrt
  d_2 = d_1 - sigma * np.sqrt(t)


In [15]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [8]:
stg = OptionStrategy(call=call, put=put,pct_daily_cp=0.25)

In [10]:
stg.covered_call()

,ua,ua_final,t,bs,strategy,position,max_pot_profit,max_pot_loss,break_even,current_profit,pct_cp,pct_daily_cp
0,خودرو,3721,27,-152,CoveredCall,"buy=خودرو, buy_at=3721, sell=ضخود2036, sell_at...",409,-3591,3591,409,11.39,0.42
2,خودرو,3721,27,-152,CoveredCall,"buy=خودرو, buy_at=3721, sell=ضخود2036, sell_at...",469,-3531,3531,469,13.28,0.49
3,خودرو,3721,27,-152,CoveredCall,"buy=خودرو, buy_at=3721, sell=ضخود2036, sell_at...",509,-3491,3491,509,14.58,0.54
4,خودرو,3721,27,-152,CoveredCall,"buy=خودرو, buy_at=3721, sell=ضخود2036, sell_at...",519,-3481,3481,519,14.91,0.55
30,خودرو,3721,118,270,CoveredCall,"buy=خودرو, buy_at=3721, sell=ضخود5021, sell_at...",929,-3071,3071,929,30.25,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...
2184,خپارس,1636,66,500,CoveredCall,"buy=خپارس, buy_at=1636, sell=ضخپارس300, sell_a...",182,-1018,1018,182,17.88,0.27
2185,خپارس,1636,66,500,CoveredCall,"buy=خپارس, buy_at=1636, sell=ضخپارس300, sell_a...",202,-998,998,202,20.24,0.31
2189,خپارس,1636,66,415,CoveredCall,"buy=خپارس, buy_at=1636, sell=ضخپارس301, sell_a...",264,-1036,1036,264,25.48,0.39
2232,پالایش,174400,94,-3154,CoveredCall,"buy=پالایش, buy_at=174400, sell=ضپلا4021, sell...",65612,-174388,174388,65612,37.62,0.40


In [56]:
class OptionStrategy:
    def __init__(self, call:pd.DataFrame, put:pd.DataFrame, pct_daily_cp:float=None) -> None:
            self.call = call
            self.put = put
            self.pct_daily_cp = pct_daily_cp
            self.rep_df = pd.DataFrame(columns=["strategy","position","ua", "ua_final", "t", "bs", "max_pot_profit", "max_pot_loss", "break_even", "current_profit", "pct_cp", "pct_daily_cp", "evaluation"]) 
    def rep_columns(self, cols):
         return [i for i in cols if i in self.rep_df.columns]
    
    def position(self, df):
        def replace_(t:str, char):
            for i in char:
                t = t.replace(i, "")
            return t
        df[["strategy","position"]] = df.stg.map(str).str.split("(", expand=True)
        df["position"] = df.position.apply(lambda x: replace_(x, ["'", ")"]))
        return df


    def married_put(self, df: pd.DataFrame):
        """
        خرید داراییِ پایه و خریدِ همزمان اختیارِ فروش
        سود-زیان  برابرِ اختلافِ ارزشِ روز دارایی (در صورت کاهش تا حد قیمتِ اعمال) با ارزشِ خرید دارایی منهایِ ارزش خرید اختیارِ فروش است.
        :param df: with (ua, symbol, strike_price, ua_sell_price, sell_price) columns. ua: Underlying Asset
        :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
                break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
                pct_max_profit :
                current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
                pct_current_profit :
        """
        df = self.put[self.put.sell_price > 0].copy()
        stg = namedtuple("MarriedPut", "buy buy_at buy_ buy_at_")
        df["stg"] = df.apply(lambda x: stg(buy=x["ua"],buy_at=x["ua_final"],  buy_=x["option"], buy_at_=x["buy_price"]), axis=1)
        df = self.position(df)

        df = df.assign(max_pot_profit="indefinite")
        df = df.assign(break_even=df.ua_sell_price + df.sell_price)
        df["current_profit"] = df.apply(
            lambda x: max(x["strike_price"], x["ua_sell_price"])
            - x["ua_sell_price"]
            - x["sell_price"],
            axis=1,
        )
        df = df.assign(pct_cp=df.current_profit / df.break_even * 100).round(2)
        df = df.assign(pct_daily_cp=df.pct_cp / df.t,
                       strategy1 = inspect.stack()[0][3])

        return df[self.rep_columns(df.columns)]
    

In [111]:
test = test.replace(np.nan, None)
test.head()

,strategy,option,ua,t,bs,max_pot_profit,max_pot_loss,break_even,current_profit,pct_cp,pct_daily_cp
0,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,608,None,3392,190,None,None
3,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,588,None,3412,170,None,None
4,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,595,None,3405,177,None,None
5,"CoveredCall(buy='(خودرو:3543)', sell='ضخود5017')",ضخود5017,خودرو,124,816,428,None,2572,428,None,None
6,"CoveredCall(buy='(خودرو:3543)', sell='ضخود5017')",ضخود5017,خودرو,124,816,322,None,2678,322,None,None


In [4]:
ri = RetailInstitutionalPower(ticker="غالبر", start_date="1401-01-01", end_date="1402-01-10")
df_ri = ri.ri_power()

In [5]:
df_ri["ticker"] = "غالبر"
df_ri = df_ri.replace(np.nan, 0)
df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]] = df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].astype(int)
df_ri = df_ri[["date", "ticker", "volume", "adj_final", "r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].rename(columns={"adj_final": "close"})
df_ri

,date,ticker,volume,close,r_buyer_power,r_seller_power,i_buyer_power,i_seller_power
0,2022-03-26,غالبر,744021,8751,7440,7223,0,0
1,2022-03-27,غالبر,892529,8840,10028,10884,0,0
2,2022-03-28,غالبر,1424034,8948,11672,15312,0,0
3,2022-03-29,غالبر,1300703,8958,9290,13204,0,6618
4,2022-03-30,غالبر,1241931,8909,8745,10706,0,0
...,...,...,...,...,...,...,...,...
228,2023-03-25,غالبر,1753975,41730,4451,8063,0,84905
229,2023-03-26,غالبر,1614018,40660,3554,5115,341,6500
230,2023-03-27,غالبر,1113640,40340,3296,5623,15881,17087
231,2023-03-28,غالبر,3498518,39980,6105,6360,203470,127243


In [8]:
db.insert_data(table="ta_ripower", df=df_ri)

In [7]:
db.drop_all(table="ta_ripower")